In [ ]:
import tensorflow as tf

2023-09-05 15:29:47.211783: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 15:29:47.271072: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-05 15:29:47.271881: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 15:29:48.482153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Define Hyper-Parameters

In [ ]:
PREPROCESSING_ARGS = {
    'sampling_rate': 16000,
    'frame_length_in_s': 0.04,
    'frame_step_in_s': 0.02,
    'num_mel_bins': 40,
    'lower_frequency': 20,
    'upper_frequency': 4000,
}

TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 10
}

alpha = 0.25

# Create Train/Val/Test Datasets

In [ ]:
train_ds = tf.data.Dataset.list_files('msc-train/*')
val_ds = tf.data.Dataset.list_files('msc-val/*')
test_ds = tf.data.Dataset.list_files('msc-test/*')

2022-11-11 14:52:07.483145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-11 14:52:07.483180: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-11 14:52:07.483196: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-bdbc19a1-4d38-4250-8aec-b2a48a575b76): /proc/driver/nvidia/version does not exist
2022-11-11 14:52:07.483469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 14:52:08.009751: W tensorflow_io/core/kernels/audio_v

TensorShape([49, 321])

In [ ]:
from preprocessing import LABELS
from preprocessing import AudioReader
from preprocessing import MelSpectrogram


audio_reader = AudioReader(tf.int16, 16000)
mel_spec_processor = MelSpectrogram(**PREPROCESSING_ARGS)

def prepare_for_training(feature, label):
    feature = tf.expand_dims(feature, -1)
    label_id = tf.argmax(label == LABELS)

    return feature, label_id


batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

train_ds = (train_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size)
            .cache())
val_ds = (val_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))
test_ds = (test_ds
            .map(audio_reader.get_audio_and_label)
            .map(mel_spec_processor.get_mel_spec_and_label)
            .map(prepare_for_training)
            .batch(batch_size))

2023-09-05 15:29:59.884693: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-09-05 15:29:59.884857: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX AVX2 AVX512F FMA


In [ ]:
for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

Batch Shape: (20, 49, 40, 1)
Data Shape: (49, 40, 1)
Labels: tf.Tensor([6 7 2 3 1 1 6 5 3 1 6 7 1 1 5 4 7 0 7 0], shape=(20,), dtype=int64)
2023-09-05 15:30:57.823796: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Create the Model


Conv2D(filters=128, kernel_size=[3, 3], stride=[2, 2], use_bias=False, padding=’valid’)
BatchNormalization()
ReLU()
Conv2D(filters=128, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding=’same’)
BatchNormalization()
ReLU()
Conv2D(filters=128, kernel_size=[3, 3], stride=[1, 1], use_bias=False, padding=’same’)
BatchNormalization()
ReLU()
GlobalAveragePooling2D()
Dense(units=8)
Softmax()


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[2, 2], use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * alpha), kernel_size=[3, 3], strides=[1, 1], use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 19, 32)        288       
                                                                 
 batch_normalization (Batch  (None, 24, 19, 32)        128       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 24, 19, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 19, 32)        9216      
                                                                 
 batch_normalization_1 (Bat  (None, 24, 19, 32)        128       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 24, 19, 32)        0

# Train the Model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
320/320 [==============================] - 124s 383ms/step - loss: 1.9460 - sparse_categorical_accuracy: 0.2297 - val_loss: 2.3244 - val_sparse_categorical_accuracy: 0.1625
Epoch 2/10
320/320 [==============================] - 32s 101ms/step - loss: 1.5094 - sparse_categorical_accuracy: 0.4352 - val_loss: 1.2631 - val_sparse_categorical_accuracy: 0.5200
Epoch 3/10
320/320 [==============================] - 32s 101ms/step - loss: 1.0474 - sparse_categorical_accuracy: 0.6311 - val_loss: 1.3135 - val_sparse_categorical_accuracy: 0.4988
Epoch 4/10
320/320 [==============================] - 33s 103ms/step - loss: 0.7905 - sparse_categorical_accuracy: 0.7337 - val_loss: 1.1000 - val_sparse_categorical_accuracy: 0.6025
Epoch 5/10
320/320 [==============================] - 34s 108ms/step - loss: 0.6544 - sparse_categorical_accuracy: 0.7837 - val_loss: 0.8035 - val_sparse_categorical_accuracy: 0.7138
Epoch 6/10
320/320 [==============================] - 32s 101ms/step - loss: 0.5634 

In [ ]:
history.history

{'loss': [1.9460313320159912,
  1.5094166994094849,
  1.047416090965271,
  0.7904614210128784,
  0.6544387936592102,
  0.5634328722953796,
  0.5053243041038513,
  0.4616104066371918,
  0.42705845832824707,
  0.40061894059181213],
 'sparse_categorical_accuracy': [0.22968749701976776,
  0.4351562559604645,
  0.6310937404632568,
  0.7337499856948853,
  0.7837499976158142,
  0.8151562213897705,
  0.8337500095367432,
  0.8510937690734863,
  0.8626562356948853,
  0.8726562261581421],
 'val_loss': [2.324437141418457,
  1.263105034828186,
  1.3135316371917725,
  1.100049376487732,
  0.8035047650337219,
  0.7267595529556274,
  0.5610030293464661,
  0.5145642161369324,
  0.5168039202690125,
  0.4528101682662964],
 'val_sparse_categorical_accuracy': [0.16249999403953552,
  0.5199999809265137,
  0.4987500011920929,
  0.6025000214576721,
  0.7137500047683716,
  0.7237499952316284,
  0.8174999952316284,
  0.8387500047683716,
  0.831250011920929,
  0.862500011920929]}

# Test the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)

40/40 [==============================] - 12s 290ms/step - loss: 0.4057 - sparse_categorical_accuracy: 0.8838


In [ ]:
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.4006
Training Accuracy: 87.27%

Validation Loss: 0.4528
Validation Accuracy: 86.25%

Test Loss: 0.4057
Test Accuracy: 88.38%


# Save the Model

### Save Keras Model

In [ ]:
import os
from time import time

timestamp = int(time())

saved_model_dir = f'./saved_models/{timestamp}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/1693928331/assets
INFO:tensorflow:Assets written to: ./saved_models/1693928331/assets


### Save Hyper-Parameters and Results

In [ ]:
import pandas as pd

output_dict = {
    'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    **TRAINING_ARGS,
    'alpha': alpha,
    'test_accuracy': test_accuracy
}

df = pd.DataFrame([output_dict])

output_path='./spectrogram_ws_results.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f1fd91f-a434-4542-983d-3ce5ae14ac33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>